Our goal in this notebook is to leverage the k-NN classifier to attempt to recognize each of cat/dog/panda species in an image using only the raw pixel intensities (i.e., no feature extraction is taking place). As we’ll see, raw pixel intensities do not lend themselves well to the k-NN algorithm. Nonetheless, this is an important benchmark experiment to run so we can appreciate why Convolutional Neural Networks are able to obtain such high accuracy on raw pixel intensities while traditional machine learning algorithms fail to do so.

**A basic Image Preprocessor**

In [4]:
import cv2 as cv
import numpy as np
import os

In [7]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/first-image-classifier/first-image-classifier.zip

--2023-06-03 11:10:06--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/first-image-classifier/first-image-classifier.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.92.165.26, 52.92.196.178, 52.92.213.2, ...
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.92.165.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196971354 (188M) [application/zip]
Saving to: ‘first-image-classifier.zip’

first-image-classif 100%[===================>] 187.85M   954KB/s    in 3m 11s  

2023-06-03 11:13:19 (1006 KB/s) - ‘first-image-classifier.zip’ saved [196971354/196971354]



In [8]:
!unzip -qq first-image-classifier.zip

In [5]:
class SimplePreprocessor:
    
    def __init__(self,width,hieght,inter=cv.INTER_AREA):
        # store the target image width, height, and interpolation method used when resizing
        self.width = width
        self.hieght = hieght
        self.inter = inter
        
    def preprocess(self,image):
        # resize the image to a fixed size, ignoring the aspect ratio
        return cv.resize(image,(self.width,self.hieght),interpolation=self.inter)
    

**Building an Image Loader**

In [48]:
class SimpleDatasetLoader:
    def __init__(self,preprocessors=None):
        # store the image preprocessor
        self.preprocessors = preprocessors
        # if the preprocessors are None, initialize them as an empty list
        if self.preprocessors is None:
            self.preprocessors = []
    def load(self,imagePaths,verbose = -1):
        
        data = []
        labels = []
        
        for (i,imagePath) in enumerate (imagePaths):
            # load the image and extract the class label assuming
            # that our path has the following format:
            # /path/to/dataset/{class}/{image}.jpg
            image = cv.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            
            if self.preprocessors is not None:
            # loop over the preprocessors and apply each to
            # the image
                for p in self.preprocessors:
                    image = p.preprocess(image)
            
            # treat our processed image as a "feature vector"
            # by updating the data list followed by the labels
            data.append(image)
            labels.append(label)
            
            
            if verbose > 0 and i > 0 and (i+1) % verbose ==0:
                # show an update every `verbose` images
                print("[INFO] processed {}/{}".format(i + 1,len(imagePaths)))
                
        # return a tuple of the data and labels
        return (np.array(data), np.array(labels))

As you can see, our dataset loader is simple by design; however, it affords us the ability to apply any number of image preprocessors to every image in our dataset with ease. The only caveat of this dataset loader is that it assumes that all images in the dataset can fit into main memory at once.

For datasets that are too large to fit into your system’s RAM, we’ll need to design a more complex dataset loader.

**k-NN: A Simple Classifier**

The k-Nearest Neighbor classifier is by far the most simple machine learning and image classification algorithm. In fact, it’s so simple that it doesn’t actually “learn” anything. Instead, this algorithm directly relies on the distance between feature vectors (which in our case, are the raw RGB pixel intensities of the images).

Simply put, the k-NN algorithm classifies unknown data points by finding the most common class among the k closest examples. Each data point in the k closest data points casts a vote, and the category with the highest number of votes wins.

**Remark**: In the event of a tie, the k-NN algorithm chooses one of the tied class labels at random

**k-NN Hyperparameters**

There are two clear hyperparameters that we are concerned with when running the k-NN algorithm. The first is obvious: the value of k. What is the optimal value of k? If it’s too small (e.g., k = 1), then we gain efficiency but become susceptible to noise and outlier data points. However, if k is too large, then we are at risk of over-smoothing our classification results and increasing bias.
The second parameter we should consider is the actual distance metric. 

In [14]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths

In [56]:
args = {
    "dataset": "./first-image-classifier/dataset/animals/",
    "neighbors": 3,
    "jobs": -1
}

In [52]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args['dataset']))

sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.reshape((data.shape[0], 3072))

# show some information on memory consumption of the images
print("[INFO] features matrix: {:.1f}MB".format(
    data.nbytes / (1024 * 1024.0)))

[INFO] loading images...
[INFO] processed 500/3000
[INFO] processed 1000/3000
[INFO] processed 1500/3000
[INFO] processed 2000/3000
[INFO] processed 2500/3000
[INFO] processed 3000/3000
[INFO] features matrix: 8.8MB


In [53]:
# encode the labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.25, random_state=42)

In [57]:
# train and evaluate a k-NN classifier on the raw pixel intensities
print("[INFO] evaluating k-NN classifier...")
model = KNeighborsClassifier(n_neighbors=args["neighbors"],
    n_jobs=args["jobs"])
model.fit(trainX, trainY)
print(classification_report(testY, model.predict(testX),
    target_names=le.classes_))

[INFO] evaluating k-NN classifier...
              precision    recall  f1-score   support

        cats       0.39      0.62      0.48       239
        dogs       0.42      0.47      0.44       262
       panda       0.92      0.29      0.44       249

    accuracy                           0.46       750
   macro avg       0.58      0.46      0.45       750
weighted avg       0.58      0.46      0.45       750



**Pros and Cons of k-NN**

One main advantage of the k-NN algorithm is that it’s extremely simple to implement and understand. Furthermore, the classifier takes absolutely no time to train, since all we need to do is store our data points for the purpose of later computing distances to them and obtaining our final classification.

However, we pay for this simplicity at classification time. Classifying a new testing point requires a comparison to every single data point in our training data, which scales O(N), making working with larger datasets computationally prohibitive.

Finally, the k-NN algorithm is more suited for low-dimensional feature spaces (which images are not). Distances in high-dimensional feature spaces are often unintuitive.

It’s also important to note that the k-NN algorithm doesn’t actually “learn” anything — the algorithm is not able to make itself smarter if it makes mistakes; it’s simply relying on distances in an n-dimensional space to make the classification.